In [2]:
from dataclasses import dataclass
import re

In [3]:
@dataclass
class GoodMove:
    cards: set#[str]
    action: str
    pot_size: int

    def __str__(self):
        return "{pot size: "+str(self.pot_size)+"; "+str(self.cards)+" => "+self.action+"}"

    def __repr__(self):
        return "{pot size: "+str(self.pot_size)+"; "+str(self.cards)+" => "+self.action +"}"

print([GoodMove(set(["Ac","As"]),"raises", 100),GoodMove(set(["Tc","Js"]),"raises", 150)])

[{pot size: 100; {'As', 'Ac'} => raises}, {pot size: 150; {'Js', 'Tc'} => raises}]


In [9]:
# input gamestate:
# each card using a 1hot-boolean vector for suit and a 1hot-boolean vector for rank
# give it the river and the hole as tensors
# size of the pot
# opponent chip contribution in previous round

with open('training_log.txt', 'r') as f:
    gamelog = f.read()
games = gamelog.split("\n\n")

bracket_matcher = re.compile("\[(.*?)\]")
parenthesis_matcher = re.compile("\((.*?)\)")

def parse_game(game:str): #-> list[GoodMove]:
    #print(game)
    good_moves = []
    gamelines = game.split('\n')

    A_moves = list(filter(lambda line: line[0]=='A', gamelines))
    B_moves = list(filter(lambda line: line[0]=='B', gamelines))
    player_moves = {'A': A_moves, 'B': B_moves}
    
    dealer_moves = list(filter(lambda line: not (line in A_moves or line in B_moves), gamelines))

    A_awarded = int(A_moves[-1].split()[-1])
    winner = 'A' if A_awarded > 0 else 'B'
    
    
    winning_hole = set()
    losing_hole = set()
    river = set()
    winning_action = None
    pot_size = 2
    opp_contribution = 0
    for line in gamelines:

        winning_turn = line[0] == winner


        if "dealt" in line:
            cards = bracket_matcher.search(line).group().replace("[", "").replace("]","").split()
            if winning_turn:
                winning_hole = set(cards)
            else:
                losing_hole = set(cards)
            continue

        if "Flop" in line or "Turn" in line or "River" in line or "Run" in line:
            cards = bracket_matcher.search(line).group().replace("[","").replace("]","").split()
            river = river.union(set(cards))
            pot_size = int(parenthesis_matcher.search(line).group().replace("(", "").replace(")",""))
            continue

        if winning_turn:
            if "calls" in line or "checks" in line or "bets" in line or "raises" in line:
                winning_action = line.split()[1]
                winning_move = GoodMove(winning_hole.union(river), winning_action, pot_size)
                good_fold = GoodMove(losing_hole.union(river), "folds", pot_size)            
                
                good_moves.append(winning_move)
                #good_moves.append(good_fold) ### commented out for testing removing folds from dataset
    
    return good_moves


parsed_moves = []
for game in games[1:-1]:
    parsed_moves+=parse_game(game)

print(parsed_moves[:10], len(parsed_moves)) 


[{pot size: 2; {'As', 'Qs'} => raises}, {pot size: 2; {'As', 'Qs'} => raises}, {pot size: 23; {'6h', 'Kd', '3h', 'Qs', 'As'} => checks}, {pot size: 23; {'6h', 'Kd', '3h', 'Qs', 'Tc', 'As'} => bets}, {pot size: 57; {'6h', 'Kd', 'Ac', '3h', 'Qs', 'Tc', 'As'} => checks}, {pot size: 2; {'Th', '5c'} => checks}, {pot size: 2; {'Th', '5d', '5c', 'Ac', '9s'} => bets}, {pot size: 2; {'Qc', '3d'} => calls}, {pot size: 2; {'Qc', '3d'} => calls}, {pot size: 4; {'Qc', 'Qs', '9d', '3d', '2c'} => checks}] 5060


In [13]:
# action order: fold, call, check, raise
import numpy as np
import eval7
action_encodings = {"folds": 0, "calls": 1, "checks":2, "raises": 3, "bets": 3} # bet is the same as raise
nofold_action_encoding = {"calls":0, "checks":1, "raises": 2, "bets":2}
MAX_CARDS = 15
def card_strs2vec(cards):#:list[str]):
    card_encodings = np.zeros((4+13)*MAX_CARDS)
    
    for card_idx, card_str in enumerate(cards):
        card = eval7.Card(card_str)
        #print("card.rank", card.rank, "card.suit", card.suit)
        offset = (4+13)*card_idx

        card_encodings[offset + card.suit] = 1
        card_encodings[offset + 4 + card.rank] = 1
    return card_encodings

def goodmove2vec(move:GoodMove):
    action_encoding = nofold_action_encoding[move.action]#action_encodings[move.action] # xgboost doesn't use 1hot encoding for classes
    card_encodings = card_strs2vec(move.cards)
    return np.concatenate((card_encodings, [move.pot_size])), action_encoding

In [7]:
print(goodmove2vec(parsed_moves[2]))

(array([1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0

In [16]:
X, Y = zip(*[goodmove2vec(move) for move in parsed_moves])
X = np.stack(X)
Y = np.stack(Y)

In [24]:
from collections import Counter
c = Counter(Y)
print(c)

Counter({2: 2498, 1: 1362, 0: 1200})


In [10]:
!python -m pip install xgboost

     --------------------------------------- 89.1/89.1 MB 31.2 MB/s eta 0:00:00


In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)
print(xtrain[:10], ytrain[:10])

[[0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 2.]
 [0. 1. 0. ... 0. 0. 2.]
 ...
 [0. 0. 1. ... 0. 0. 4.]
 [0. 0. 1. ... 0. 0. 2.]
 [1. 0. 0. ... 0. 0. 4.]] [2 0 0 0 1 2 1 1 0 0]


In [12]:
xgbc = xgb.XGBClassifier(max_depth = 5, n_estimators = 800, objective='multi:softmax', num_class = 4)
xgbc.fit(xtrain, ytrain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=800, n_jobs=None, num_class=4,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [13]:
ypred = xgbc.predict(xtest)
mse = mean_squared_error(ytest, ypred)
acc = accuracy_score(ytest, ypred)
print("RMSE: %.2f" % (mse**(1/2.0)) + "% Accuracy:", str(acc*100)+"%" ) 

RMSE: 1.56% Accuracy: 46.701846965699204%


In [16]:
param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
    #'updater': ['grow_gpu']
}
from sklearn.model_selection import GridSearchCV
grid_cv = GridSearchCV(xgbc, param_grid, n_jobs=-1, cv=3, scoring="accuracy")

In [17]:
_ = grid_cv.fit(xtrain, ytrain)

In [19]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'reg_lambda': 10, 'subsample': 0.8}
0.49174917491749176


In [18]:
best_params = {'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'reg_lambda': 10, 'subsample': 0.8}
xgbc = xgb.XGBClassifier( n_estimators = 800, objective='multi:softmax', num_class = 3, **best_params)
xgbc.fit(xtrain, ytrain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=800, n_jobs=None, num_class=3,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [19]:
ypred = xgbc.predict(xtest)
mse = mean_squared_error(ytest, ypred)
acc = accuracy_score(ytest, ypred)
print("RMSE: %.2f" % (mse**(1/2.0)) + "% Accuracy:", str(acc*100)+"%" ) 

RMSE: 0.95% Accuracy: 57.70750988142292%


In [22]:
testhand = ['As', '7c', '2d','Tc',"3d"]
print(testhand)
pred_class = xgbc.predict([card_strs2vec(testhand)])[0]
print(["call", "check", "raise"][pred_class])

['As', '7c', '2d', 'Tc', '3d']
raise


In [22]:
import pickle
file_name = "xgb_model.pkl"

# save
pickle.dump(xgbc, open(file_name, "wb"))


In [8]:
import xgboost as xgb
import pickle
file_name = "xgb_model.pkl"

xgbc_model = pickle.load(open(file_name, "rb"))
testhand = ['As', 'Ac', 'Ad','Tc',"3d"]
print(testhand)
pred_class = xgbc_model.predict([card_strs2vec(testhand)])[0]
print(["fold", "call", "check", "raise"][pred_class])

['As', 'Ac', 'Ad', 'Tc', '3d']
fold
